<a href="https://colab.research.google.com/github/XieKaixuan/AdaBoost-SVM/blob/master/notebooks/datacommons_mcp_tools_with_custom_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copyright 2025 Google LLC. SPDX-License-Identifier: Apache-2.0

---
# ✨ Explore Data Commons MCP Tools with a Custom Agent
This notebook demonstrates how to build and run an agent using Google's [Agent Development Kit (ADK)](https://google.github.io/adk-docs/). The agent you'll build communicates over HTTP with a separate process running the [Data Commons MCP tools](https://pypi.org/project/datacommons-mcp/).

### ⚠️ Important notes
* **Alternative implementation:** For an example of an agent using the ADK's native web framework with `stdio`, see the [Data Commons MCP sample agents](https://github.com/datacommonsorg/agent-toolkit/blob/main/docs/user_guide.md#use-the-sample-agent).

* **Resource limits:** This Colab environment has resource constraints and is best suited for simple queries. For more data-intensive tasks, please use the [Gemini CLI](https://github.com/datacommonsorg/agent-toolkit/blob/main/docs/quickstart.md) or another agent framework.






---



## Create your ADK agent

#### **Step 0**: Make a copy of this notebook:
* Go to **File** > **Save a copy in Drive**.


#### **Step 1**: Add your Data Commons and Gemini API keys to the notebook's secrets:

1. Open the **Secrets** panel by clicking the key icon (key.png) in the left menu.
1. Click **+ Add new secret** and set the `Name` and `Value` fields for each required API key listed below:
   - Data Commons API Key:
     - **Name**: `DC_API_KEY`
     - **Value**: Your Data Commons API key obtained from https://apikeys.datacommons.org/
   - Gemini API Key:
     - **Name**: `GOOGLE_API_KEY` or `GEMINI_API_KEY`
     - **Value**: A Google Gemini-enabled API key obtained from https://aistudio.google.com/apikey
1. Enable **Notebook access** for both.

In [22]:
# @title #### **Step 2.** Import required libraries and initialize secrets. {"display-mode":"form"}
# @markdown 👉 Run this cell

!pip uninstall -q -y google-genai google-adk
!pip install -q google-genai==1.36.0 google-adk==1.14.1

import logging
import os
import warnings
from google.colab import userdata


logger = logging.getLogger()
logger.setLevel(logging.ERROR)

warnings.filterwarnings(
    "ignore", category=UserWarning, module="google.adk.tools.mcp_tool"
)

GOOGLE_API_KEY=""
DC_API_KEY=""

# Get and validate DC_API_KEY
try:
    DC_API_KEY = userdata.get("DC_API_KEY")
except userdata.SecretNotFoundError:
    raise ValueError(
        "`DC_API_KEY` not found in Colab secrets. "
        "Please follow the instructions in 'Step 1' to add the secret."
    ) from None
os.environ["DC_API_KEY"] = DC_API_KEY

# Get and validate the Google/Gemini API key
try:
    GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
except userdata.SecretNotFoundError:
    try:
        GOOGLE_API_KEY = userdata.get("GEMINI_API_KEY")
    except userdata.SecretNotFoundError:
        raise ValueError(
            "Could not find `GOOGLE_API_KEY` or `GEMINI_API_KEY` in Colab secrets. "
            "Please follow the instructions in 'Step 1' to set one of these secrets."
        ) from None

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

print("✅ API keys successfully loaded and set.")

✅ API keys successfully loaded and set.


In [23]:
# @title #### **Step 3.** Start the Data Commons MCP server in a subprocess. {"display-mode":"form"}
# @markdown 👉  Run this cell

import asyncio
import subprocess
import sys
import time

import aiohttp

MCP_PORT = 3000


async def wait_for_server_ready(
    url: str, process: subprocess.Popen, timeout_sec: int = 15
):
    """
    Actively polls the server's HTTP endpoint AND checks that the subprocess
    is still running. Replaces the unreliable 'asyncio.sleep()'.
    """
    logger.info(f"Waiting for server to launch at {url}...")
    start_time = time.time()

    async with aiohttp.ClientSession() as session:
        while True:
            # 1. First, check if the subprocess crashed or exited
            return_code = process.poll()
            if return_code is not None:
                # The process terminated. This is a hard failure.
                # Logs (if any) will have already printed to the notebook output.
                logger.error(
                    f"Server process failed to start (exit code {return_code}). See notebook output for server logs."
                )
                raise ChildProcessError(
                    f"MCP server process terminated unexpectedly (exit code {return_code}). See notebook output for server logs."
                )

            # 2. If it's running, try to connect via HTTP
            try:
                # We just need *any* response. Even a 404 (Not Found) or 405 (Method Not Allowed)
                # proves the server is up and responding to HTTP requests.
                # aiohttp.ClientConnectorError is the "Connection Refused" we expect to see while it's starting.
                async with session.get(url):
                    print("✅ MCP Server is up and responding.")
                    return  # Success!

            except aiohttp.ClientConnectorError:
                # This is expected. Server is not ready yet. Keep waiting.
                pass

            except Exception as e:
                # Any other exception might be a problem, log it.
                logger.warning(f"Health check received unexpected error: {e}")

            # 3. Check if we've run out of time
            if time.time() - start_time > timeout_sec:
                logger.error(
                    f"Timeout: Server did not respond within {timeout_sec} seconds."
                )
                # Kill the hung process
                process.kill()
                # Logs (if any) will have already printed to the notebook output.
                logger.error("Server logs (if any) should be visible above.")
                raise TimeoutError(
                    f"MCP Server failed to start at {url} within {timeout_sec}s."
                )

            # 4. Wait a moment before the next poll
            await asyncio.sleep(0.5)  # Poll every 500ms


#
# --- Install uv in subprocess env ---
#
logger.info("Installing 'uv' package...")
install_process = subprocess.run(
    [
        sys.executable,
        "-m",
        "pip",
        "install",
        "-q",
        "uv",
    ],  # This is the correct way to run pip
    check=False,
    capture_output=True,
    text=True,
)

if install_process.returncode != 0:
    logger.error("Subprocess failed to install 'uv': %s", install_process.stderr)
    raise RuntimeError("Could not install uv on subprocess")
else:
    logger.info("'uv' package successfully installed.")

#
# --- Kill any existing process on the MCP port ---
#
logger.info(f"Ensuring port {MCP_PORT} is free...")
# Kill any process on the port; we don't need to see the output.
subprocess.run(
    ["fuser", "-k", f"{MCP_PORT}/tcp"],
    check=False,
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL,
)

#
# ---Start MCP Server ---
#
logger.info("Starting MCP server...")
kernel_env = os.environ.copy()
mcp_server_process = subprocess.Popen(
    [
        sys.executable,  # Call Python
        "-m",
        "uv",  # Run the 'uv' module (NOT 'uvx' or the install string)
        "tool",
        "run",  # The rest of the uv command
        "datacommons-mcp",
        "serve",
        "http",
        "--port",
        str(MCP_PORT),
    ],
    env=kernel_env,
    stdout=None,
    stderr=None,
)
# Give the server a moment to start up, then check that the server is running
await asyncio.sleep(2)
await wait_for_server_ready(
    url=f"http://localhost:{MCP_PORT}", process=mcp_server_process
)

✅ MCP Server is up and responding.


#### **Step 4**. Set up your agent.

In [24]:
# @title ##### **a.** Select a model {"display-mode":"form"}
# @markdown 👉 Select your desired agent model, then **run this cell**.

AGENT_MODEL = "gemini-2.5-pro"  # @param ["gemini-2.5-flash","gemini-2.5-pro","gemini-2.5-flash-lite"] {"allow-input":true}

print(f"✅ Agent model selected.")

✅ Agent model selected.


In [25]:
# @title ##### **b.** Define the agent's core instructions (system prompt). {"display-mode":"form"}
# @markdown 👉 Select or write your own prompt to set the agent's permanent rules and personality, then **run this cell**.
# @markdown
# @markdown > Here are some tips for writing great agent instructions: [Google ADK - Guiding the Agent: Instructions](https://google.github.io/adk-docs/agents/llm-agents/#guiding-the-agent-instructions-instruction).
AGENT_INSTRUCTIONS = "Use the Data Commons MCP tools to respond to user queries. Cite the data source when possible." # @param ["Use the Data Commons MCP tools to respond to user queries. Cite the data source when possible.","You are an agent that compares statistics between multiple locations. Use the Data Commons tools to fetch the data, present the comparison in a markdown table, and always cite the data source.","You are an agent that retrieves time-series data. When a user asks for a statistic over a range of years for a specific place, use the Data Commons tools to find the data, present it as a year-by-year list, and cite the source."] {"allow-input":true}

print("✅ Agent instructions initialized.")

✅ Agent instructions initialized.


In [26]:
# @title ##### **c.** Create the agent and initialize a session {"display-mode":"form"}
# @markdown 👉 Run this cell
# @markdown
# @markdown **Note**: You must re-run this cell every time you update the AGENT_INSTRUCTIONS or change the model!

from google.adk.agents.llm_agent import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.mcp_tool.mcp_toolset import (
    McpToolset,
    StreamableHTTPConnectionParams,
)
from google.genai import types

from dataclasses import dataclass, field

from IPython.display import Markdown, clear_output, display

#
# ---Define the Agent ---
# Key Concept: Create the agent w/ system prompt, model, and MCP tools.
#

datcom_agent = LlmAgent(
    name="datacommons_agent",
    model=AGENT_MODEL,
    instruction=AGENT_INSTRUCTIONS,
    tools=[
        McpToolset(
            connection_params=StreamableHTTPConnectionParams(
                url=f"http://localhost:{MCP_PORT}/mcp"
            )
        )
    ],
)
logging.info(f"Agent '{datcom_agent.name}' created using model '{AGENT_MODEL}'.")


#
# -----------------------------------------------------------------------------
# The rest of this cell is only required for running the agent in Colab. This
# is automatically handled when running with adk web.
# ----------------------------------------------------------------------------
#

#
# ---Session Management ---
# Key Concept: SessionService stores conversation history & state.
# InMemorySessionService is  simple, non-persistent storage for this tutorial.
#
session_service = InMemorySessionService()

# Define constants for identifying the interaction context
APP_NAME = "datacommons_app"
USER_ID = "user_1"
SESSION_ID = "session_001"

# Create the specific session where the conversation will happen
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
)
logging.info(
    f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'"
)

#
# ---Runner ---
# Key Concept: Runner orchestrates the agent execution loop.
#
runner = Runner(agent=datcom_agent, app_name=APP_NAME, session_service=session_service)
logging.info(f"Runner created for agent '{runner.agent.name}'.")

#
# ---Async Agent Interaction Function ---
#


@dataclass
class AgentTurn:
    """Holds all information for one turn of conversation."""

    user_query: str
    agent_response: str
    tool_calls: list[str] = field(default_factory=list)

    def _get_text(self) -> list[str]:
        # Build the output as a list of strings
        output = [f"\n>> 👤 User: {self.user_query}\n"]

        if self.tool_calls:
            output.append("🛠️ Tool Calls:")
            for call in self.tool_calls:
                output.append(f" - {call}")
            output.append("")  # Add a newline

        return output

    def __str__(self) -> str:
        """This method is called when you use print() on the object."""

        # Build the output as a list of strings
        output = self._get_text()
        output.append(f"<< 🤖 Agent: {self.agent_response}\n")

        # Join all the parts into a single string
        return "\n".join(output)

    def print_pretty(self) -> None:
        raw_text = self._get_text()
        print("\n".join(raw_text))
        print("<< 🤖 Agent:\n")
        display(Markdown(self.agent_response))
        print("")


def print_logs() -> None:
    for log in logs:
        log.print_pretty()


async def call_agent_async(query: str, runner, user_id, session_id) -> None:
    """Sends a query to the agent and prints the final response."""

    if logs:
        print("Session History:")
        print_logs()
        print(
            "---------------------------------------------------------\n\nCurrent Query:"
        )

    print(f"\n>> 👤 User Query: {query}")
    print("\n...waiting for agent's response\n")

    # Prepare the user's message in ADK format
    content = types.Content(role="user", parts=[types.Part(text=query)])

    final_response_text = "Agent did not produce a final response."  # Default

    # Iterate through events to find the final answer.
    tool_calls = []
    async for event in runner.run_async(
        user_id=user_id, session_id=session_id, new_message=content
    ):
        # You can uncomment the line below to see *all* events during execution
        logger.info(
            f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}"
        )
        if event.author == "datacommons_agent":
            for tool_call in event.get_function_calls():
                if not tool_calls:
                    print("🛠️ Tool Calls:")

                args = ", ".join(
                    sorted([f"{k}={repr(v)}" for k, v in tool_call.args.items()])
                )
                tool_call_str = f"{tool_call.name}({args})"
                print(f" - {tool_call_str}")
                tool_calls.append(tool_call_str)

        # Key Concept: is_final_response() marks the concluding message for the turn.
        if event.is_final_response():
            if event.content and event.content.parts:
                # Assuming text response in the first part
                final_response_text = event.content.parts[0].text
            elif (
                event.actions and event.actions.escalate
            ):  # Handle potential errors/escalations
                final_response_text = (
                    f"Agent escalated: {event.error_message or 'No specific message.'}"
                )
            # Add more checks here if needed (e.g., specific error codes)
            break  # Stop processing events once the final response is found

    log = AgentTurn(
        user_query=query, agent_response=final_response_text, tool_calls=tool_calls
    )
    clear_output(wait=True)
    if logs:
        print("Session History:")
        print_logs()
        print(
            "---------------------------------------------------------\n\nCurrent Query:\n"
        )
    log.print_pretty()
    logs.append(
        AgentTurn(
            user_query=query, agent_response=final_response_text, tool_calls=tool_calls
        )
    )


logs = []

print(f"✅ Agent created with a fresh session history.")

✅ Agent created with a fresh session history.




---
## 🚀 Take your agent for a spin!

You're all set! It's time to chat with the agent you just built.

#### **👉 How to use this cell:**

* **Type your question:** Click into the input box below.

* **Run the cell:** Press the ▶ "play" button on the left.
  * **Pro-tip:** You can also use a keyboard shortcut to run the cell, and your cursor will stay in the box, ready for your next question:
    * **Windows:** `Ctrl + Enter`
    * **Mac:** `Cmd + Enter`

* **Keep chatting:** After you get a response, you can ask a follow-up question. Just type your new question in the same box and run the cell again. The agent will remember what you talked about.

#### **Managing your chat history:**

* **To keep your conversation going:** Just continue to use **this** cell.

* **To restart the agent (and erase its memory):** Re-run the agent setup cell (Step 4c).

In [32]:
# @title {"display-mode":"form"}
Input = "世界上树最多的10个国家，从高到底"  # @param {"type":"string", "placeholder": " "}


first_run = not logs

if not Input.strip():
    print("⚠️ Provide input for the agent, then run the cell again.")
else:
    await call_agent_async(Input, runner=runner, user_id=USER_ID, session_id=SESSION_ID)

if first_run:
  print("👉 Provide a follow-up or ask a new question using the same 'Input' box above. ")

Session History:

>> 👤 User: Which states in the US have the highest smoke pollution?

🛠️ Tool Calls:
 - search_indicators(include_topics=False, places=['USA', 'California', 'New York', 'Texas', 'Florida', 'Illinois', 'Washington'], query='smoke pollution')
 - validate_child_place_types(child_place_types=['State'], parent_place_name='USA')
 - get_observations(child_place_type='State', date='latest', place_dcid='country/USA', variable_dcid='Mean_PopulationWeighted_Concentration_AirPollutant_SmokePM25')

<< 🤖 Agent:



Based on the latest available data from 2020, the states with the highest population-weighted mean concentration of smoke PM2.5 are:

1.  **Oregon**: 30.08
2.  **Washington**: 20.75
3.  **Idaho**: 13.80
4.  **Montana**: 9.46
5.  **California**: 9.17

This data is sourced from the Stanford EcoLab.



>> 👤 User: the world population by country  

<< 🤖 Agent:



Here is the population for each country in the world as of 2024, sorted from most to least populous:

| Country | Population (2024) |
|---|---|
| India | 1,450,935,791 |
| China | 1,408,975,000 |
| United States of America | 340,110,988 |
| Indonesia | 283,487,931 |
| Pakistan | 251,269,164 |
| Nigeria | 232,679,478 |
| Brazil | 211,998,573 |
| Bangladesh | 173,562,364 |
| Russia | 143,533,851 |
| Mexico | 130,861,007 |
| Ethiopia | 132,059,767 |
| Japan | 123,975,371 |
| Egypt | 116,538,258 |
| Philippines | 115,843,670 |
| Congo [DRC] | 109,276,265 |
| Vietnam | 100,987,686 |
| Iran | 91,567,738 |
| Turkey | 85,518,661 |
| Germany | 83,510,950 |
| Thailand | 71,668,011 |
| United Kingdom | 69,226,000 |
| Tanzania | 68,560,157 |
| France | 68,516,699 |
| South Africa | 64,007,187 |
| Italy | 58,986,023 |
| Kenya | 56,432,944 |
| Myanmar [Burma] | 54,500,091 |
| Colombia | 52,886,363 |
| South Korea | 51,751,065 |
| Uganda | 50,015,092 |
| Sudan | 50,448,963 |
| Spain | 48,807,137 |
| Algeria | 46,814,308 |
| Argentina | 45,696,159 |
| Iraq | 46,042,015 |
| Afghanistan | 42,647,492 |
| Canada | 41,288,599 |
| Yemen | 40,583,164 |
| Morocco | 38,081,173 |
| Ukraine | 37,860,221 |
| Angola | 37,885,849 |
| Poland | 36,554,707 |
| Uzbekistan | 36,361,859 |
| Saudi Arabia | 35,300,280 |
| Malaysia | 35,557,673 |
| Peru | 34,217,848 |
| Ghana | 34,427,414 |
| Mozambique | 34,631,766 |
| Madagascar | 31,964,956 |
| Côte d'Ivoire | 31,934,230 |
| Nepal | 29,651,054 |
| Cameroon | 29,123,744 |
| Venezuela | 28,405,543 |
| Australia | 27,204,809 |
| Niger | 27,032,412 |
| North Korea | 26,498,823 |
| Syria | 24,672,760 |
| Mali | 24,478,595 |
| Burkina Faso | 23,548,781 |
| Sri Lanka | 21,916,000 |
| Malawi | 21,655,286 |
| Zambia | 21,314,956 |
| Kazakhstan | 20,592,571 |
| Chad | 20,299,123 |
| Chile | 19,764,771 |
| Romania | 19,069,340 |
| Somalia | 19,009,151 |
| Guatemala | 18,406,359 |
| Senegal | 18,501,984 |
| Ecuador | 18,135,478 |
| Netherlands | 17,994,237 |
| Cambodia | 17,638,801 |
| Zimbabwe | 16,634,373 |
| Guinea | 14,754,785 |
| Rwanda | 14,256,567 |
| Benin | 14,462,724 |
| Burundi | 14,047,786 |
| Bolivia | 12,413,315 |
| Tunisia | 12,277,109 |
| Haiti | 11,772,557 |
| Belgium | 11,876,844 |
| South Sudan | 11,943,408 |
| Jordan | 11,552,876 |
| Dominican Republic | 11,427,557 |
| Cuba | 10,979,783 |
| Czech Republic | 10,882,164 |
| Sweden | 10,569,709 |
| Portugal | 10,701,636 |
| Papua New Guinea | 10,576,502 |
| Tajikistan | 10,590,927 |
| Greece | 10,388,805 |
| Azerbaijan | 10,202,850 |
| Israel | 9,974,400 |
| Hungary | 9,562,314 |
| Togo | 9,515,236 |
| Belarus | 9,133,712 |
| Switzerland | 9,034,102 |
| Sierra Leone | 8,642,022 |
| Laos | 7,769,819 |
| Hong Kong | 7,524,100 |
| Turkmenistan | 7,494,498 |
| Libya | 7,381,023 |
| Kyrgyzstan | 7,224,614 |
| Paraguay | 6,929,153 |
| Nicaragua | 6,916,140 |
| Serbia | 6,587,202 |
| Bulgaria | 6,444,366 |
| Congo [Republic] | 6,332,961 |
| El Salvador | 6,338,193 |
| Singapore | 6,036,860 |
| Denmark | 5,976,992 |
| Lebanon | 5,805,962 |
| Finland | 5,637,214 |
| Norway | 5,572,272 |
| Slovakia | 5,422,069 |
| Ireland | 5,380,257 |
| New Zealand | 5,338,500 |
| Central African Republic | 5,330,690 |
| Palestinian Territories | 5,289,152 |
| Costa Rica | 5,129,910 |
| Mauritania | 5,169,395 |
| Kuwait | 4,973,861 |
| Panama | 4,515,577 |
| Croatia | 3,866,300 |
| Georgia | 3,673,850 |
| Eritrea | 3,535,603 |
| Mongolia | 3,524,788 |
| Uruguay | 3,386,588 |
| Puerto Rico | 3,203,295 |
| Bosnia and Herzegovina | 3,164,253 |
| Armenia | 3,033,500 |
| Namibia | 3,030,131 |
| Lithuania | 2,888,055 |
| Jamaica | 2,839,175 |
| Qatar | 2,857,822 |
| Albania | 2,714,617 |
| Gambia | 2,759,988 |
| Botswana | 2,521,139 |
| Gabon | 2,538,952 |
| Moldova | 2,389,275 |
| Lesotho | 2,337,423 |
| Guinea-Bissau | 2,201,352 |
| Slovenia | 2,126,324 |
| Latvia | 1,862,441 |
| Equatorial Guinea | 1,892,516 |
| Macedonia [FYROM] | 1,792,179 |
| Bahrain | 1,588,670 |
| East Timor | 1,400,638 |
| Trinidad and Tobago | 1,368,333 |
| Estonia | 1,371,986 |
| Cyprus | 1,358,282 |
| Mauritius | 1,259,509 |
| Eswatini | 1,242,822 |
| Djibouti | 1,168,722 |
| Fiji | 928,784 |
| Comoros | 866,628 |
| Guyana | 831,087 |
| Bhutan | 791,524 |
| Solomon Islands | 819,198 |
| Luxembourg | 677,717 |
| Macau | 687,000 |
| Suriname | 634,431 |
| Montenegro | 623,831 |
| Malta | 574,346 |
| Maldives | 527,799 |
| Cape Verde | 524,877 |
| Brunei | 462,721 |
| Belize | 417,072 |
| Bahamas | 401,283 |
| Iceland | 404,610 |
| Vanuatu | 327,777 |
| New Caledonia | 292,639 |
| Barbados | 282,467 |
| French Polynesia | 281,807 |
| São Tomé and Príncipe | 235,536 |
| Samoa | 218,019 |
| Saint Lucia | 179,744 |
| Guam | 167,777 |
| Curaçao | 155,900 |
| Kiribati | 134,518 |
| Seychelles | 121,354 |
| Grenada | 117,207 |
| Aruba | 107,624 |
| Federated States of Micronesia | 113,160 |
| Saint Vincent and the Grenadines | 100,616 |
| Antigua and Barbuda | 93,772 |
| Isle of Man | 84,160 |
| Andorra | 81,938 |
| Cayman Islands | 74,457 |
| Dominica | 66,205 |
| Bermuda | 64,636 |
| Greenland | 56,836 |
| Faroe Islands | 54,719 |
| Saint Kitts and Nevis | 46,843 |
| American Samoa | 46,765 |
| Turks and Caicos Islands | 46,535 |
| Northern Mariana Islands | 44,278 |
| Sint Maarten | 43,350 |
| Liechtenstein | 40,197 |
| British Virgin Islands | 39,471 |
| Gibraltar | 39,329 |
| Monaco | 38,631 |
| Marshall Islands | 37,548 |
| San Marino | 33,977 |
| Saint Martin (French part) | 26,129 |
| Palau | 17,695 |
| Nauru | 11,947 |
| Tuvalu | 9,646 |

Data from the [World Bank](https://datacatalog.worldbank.org/dataset/world-development-indicators/).


---------------------------------------------------------

Current Query:


>> 👤 User: 世界上树最多的10个国家，从高到底

<< 🤖 Agent:



根据现有数据，无法直接按树木数量进行排名，因为这是一个极难精确统计的指标。但是，我们可以使用“森林面积”作为衡量标准，这通常是评估一个国家森林资源的主要方式。

根据联合国粮食及农业组织（FAO）2020年的数据，以下是世界上森林面积最广的10个国家，从高到低排名：

| 排名 | 国家 | 森林面积 (平方公里) |
|---|---|---|
| 1 | 俄罗斯 | 8,153,116 |
| 2 | 巴西 | 4,966,196 |
| 3 | 加拿大 | 3,469,281 |
| 4 | 美国 | 3,097,950 |
| 5 | 中国 | 2,199,781 |
| 6 | 澳大利亚 | 1,340,051 |
| 7 | 刚果民主共和国 | 1,261,643 |
| 8 | 印度尼西亚 | 921,332 |
| 9 | 秘鲁 | 723,303 |
| 10 | 印度 | 721,600 |

*数据来源: [联合国粮食及农业组织 (FAO)](https://www.fao.org/forest-resources-assessment/2020/en)*


---


#### **Here are some ideas to get you started:**

* Which countries have the lowest GINI index?

* Tell me about the economy in Brazil.

* Which states in the US have the highest smoke pollution?

* How does life expectancy vary across countries in Africa?

---

 ⚠️ In case of error, click `Runtime` > `Disconnect and delete runtime` in the Menu Bar and re-run the cells.

❗ AI applications using the MCP server can make mistakes, so please double-check responses.

